# Metaclasses

Exploration together with: https://stackoverflow.com/questions/100003/what-are-metaclasses-in-python

In [1]:
class ObjectCreator:
    pass

In [2]:
my_object = ObjectCreator()

In [3]:
my_object

class statement creates an object (**the class**) in memory that itself is able to create other objects (**the instances**)

In [4]:
repr(ObjectCreator)

"<class '__main__.ObjectCreator'>"

In [5]:
repr(ObjectCreator())

'<__main__.ObjectCreator object at 0x103a2a070>'

## Creating classes dynamically

Classes are objects, so you can create them dynamically

In [6]:
def create_class(name):
    if name == 'foo':
        class Foo:
            pass
        return Foo
    else:
        class Bar:
            pass
        return Bar

In [7]:
MyClass = create_class('foo')

In [8]:
repr(MyClass)

"<class '__main__.create_class.<locals>.Foo'>"

Python creates class objects when you use `class` keyword, but there is another way to do it.

In [9]:
type(1)

int

In [10]:
type('1')

str

In [11]:
type(MyClass)

type

In [12]:
type(MyClass())

__main__.create_class.<locals>.Foo

When used with one argument `type` returns the type of an object. However, if we use it with three arguments, it returns a new type object.

```python
type(name, bases, attrs)
```

* name - name of the class. Becomes the `__name__` attribute
* bases - tuple of the parent class (for inheritance, can be empty). Becomes the `__bases__` attribute
* attrs - dictionary containing attributes. Becomes the `__dict__` attribute.

For exmaple

In [13]:
class MyClass():
    pass

can be created manually this way:

In [14]:
MyClass = type('MyClass', (), {})

In [15]:
MyClass

__main__.MyClass

`type` accepts a dictionary to define the attributes of the class

In [16]:
class Foo:
    bar = True

In [17]:
Foo.bar

True

can be translated to:

In [18]:
Foo = type('Foo', (), {'bar': True})

In [19]:
Foo.bar

True

Inheritence works as excepted

In [20]:
class FooChild(Foo):
    pass

In [21]:
FooChild.bar

True

In [22]:
FooChild = type('FooChild', (Foo,), {})

In [23]:
FooChild.bar

True

One of the more useful things for classes is adding methods. Just define a function with a proper signature and assign it as an attribute.

In [24]:
def echo_bar(self):
    print(self.bar)

In [25]:
FooChild = type('FooChild', (Foo,), {'echo_bar': echo_bar})

In [26]:
hasattr(Foo, 'echo_bar')

False

In [27]:
hasattr(FooChild, 'echo_bar')

True

In [28]:
my_foo = FooChild()
my_foo.echo_bar()

True


In Python, classes are objects, and you can create a class on the fly, dynamically.
This is what Python does when you use the keyword `class`, and it does so by using **metaclass**

## Metaclasses

Metaclasses are the 'stuff' that creates classes. They are classes' classes

`type` allows you to create classes, because `type` function is the metaclass. Python uses it to create all classes behind the scenes.

In [29]:
age = 35
age.__class__

int

In [30]:
name = 'bob'
name.__class__

str

In [31]:
def foo(): pass
foo.__class__

function

In [32]:
class Bar: pass
Bar().__class__

__main__.Bar

What is the `__class__` of any `__class__`?

In [33]:
age.__class__.__class__

type

In [34]:
name.__class__.__class__

type

In [35]:
foo.__class__.__class__

type

In [36]:
Bar().__class__.__class__

type

A metaclass is just the object that creates class objects.

`type` is the built-in metaclass Python uses,but you can create your own metaclass.

Imagine an example, where you decide that all classes in your module should have their attributes written in uppercase.
There are several ways to do this, but one way is to set `__metaclass__` at the module level.

Luckily, `__metaclass__` can actually be any callable, it doesn't need to be a formal class (somehting with 'class' in its name doesn't need to be a class_

In [37]:
# %load metaclasses.py
# the metaclass will automatically get passed the same argument
# that you usually pass to `type`

def upper_attr(future_class_name, future_class_parents, future_class_attrs):
    """
    Return a class object, with the list of its attributes turned into uppercase
    """
    # pick any attribute that doesn't start with '__' and uppercase it
    uppercase_attrs = {
        attr if attr.startswith('__') else attr.upper(): v
        for attr, v in future_class_attrs.items()
    }
    # let `type` do the class creation
    print(uppercase_attrs)
    return type(future_class_name, future_class_parents, uppercase_attrs)

__metaclass__ = upper_attr # this will affect all classes in the module

class Foo():
    bar = 'baz'
    
print(hasattr(Foo, 'bar'))
print(hasattr(Foo, 'BAR'))
print(Foo.BAR)

True
False


AttributeError: type object 'Foo' has no attribute 'BAR'

It only works when you execute it from file.

Now let's do exactly the same, but using a real class for a metaclass.

In [38]:
# remember that `type` is actually a class like 'str' and 'int'
# so you can inherit from it

class UpperAttrMetaclass(type):
    # __new__ is the method called before __init__
    # it's the method that creates the object and returns it
    # while __init__ just initializes the object passed as parameter
    # you rearely use __new__, except when you want to control how the object is created.
    # here the created object is the class, and we want to customize it
    # so we override __new__
    
    def __new__(upperattr_metaclass, future_class_name, future_class_parents, future_class_attrs):
        uppercase_attrs = {
            attr if attr.startswith('__') else attr.upper(): v
            for attr, v in future_class_attrs.items()
        }
        return type(future_class_name, future_class_parents, upppercase_attrs)

Let's rewritte the above, but with shorter and more realistic variable names now that we know what they mean:

In [39]:
class UpperAttrMetaClass(type):
    def __new__(cls, clsname, bases, attrs):
        uppercase_attrs = {
            attr if attr.startswith('__') else attr.upper(): v
            for attr, v in attrs.items()
        }
        return type(clsname, bases, uppercase_attrs)

You may have noticed the extra argument `cls`. There is nothing special about it: `__new__` always receives the class it's defined in, as the first parameter.
Just like you have `self` for ordinary methods which receive the instance as the first parameter, or the defining class for class methids.

But this is not proper OOP! We are calling `type` directly and we aren't overriding or calling the parent's `__new__`. Let's do that instead:

In [40]:
class UpperAttrMetaclass(type):
    def __new__(cls, clsname, bases, attrs):
        uppercase_attrs = {
            attr if attr.startswith('__') else attr.upper(): v
            for attr, v in attrs.items()
        }
        return type.__new__(cls, clsname, bases, uppercase_attrs)

We can make it even cleaner by using `super`, which will ease inheritance (because yes, you can have metaclasses, inheriting from metaclasses, inheriting from `type`) 

In [41]:
class UpperAttrMetaclass(type):
    def __new__(cls, clsname, bases, attrs):
        uppercase_attrs = {
            attr if attr.startswith('__') else attr.upper(): v
            for attr, v in attrs.items()
        }
        return super(UpperAttrMetaclass, cls).__new__(
            cls, clsname, bases, uppercase_attrs
        )